In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import tensorflow as tf
from tensorflow.keras.layers import Dense,Bidirectional,GRU,Dropout,Input, Embedding,TextVectorization, LSTM, MultiHeadAttention, LayerNormalization
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers.schedules import LearningRateSchedule
from tensorflow.keras.optimizers import Adam

In [ ]:
!wget https://www.manythings.org/anki/fra-eng.zip

In [ ]:
!unzip '/content/fra-eng.zip' -d '/content/dataset'

In [ ]:
text_dataset = tf.data.TextLineDataset('/content/dataset/fra.txt')

In [ ]:
for i in text_dataset.take(3):
  print(i)

In [ ]:
VOCAB_SIZE=20000
ENGLISH_SEQUENCE_LENGTH=64
FRENCH_SEQUENCE_LENGTH=64
EMBEDDING_DIM=300
BATCH_SIZE=64

In [ ]:
english_vectorize_layer=TextVectorization(
    standardize='lower_and_strip_punctuation',
    max_tokens=VOCAB_SIZE,
    output_mode='int',
    output_sequence_length=ENGLISH_SEQUENCE_LENGTH
)

In [ ]:
french_vectorize_layer=TextVectorization(
    standardize='lower_and_strip_punctuation',
    max_tokens=VOCAB_SIZE,
    output_mode='int',
    output_sequence_length=FRENCH_SEQUENCE_LENGTH
)

In [ ]:
def selector(input_text):
  split_text=tf.strings.split(input_text,'\t')
  return {'input_1':split_text[0:1],'input_2':'starttoken '+split_text[1:2]},split_text[1:2]+' endtoken'

In [ ]:
split_dataset=text_dataset.map(selector)

In [ ]:
for i in split_dataset.take(3):
  print(i)

In [ ]:
def separator(input_text):
  split_text=tf.strings.split(input_text,'\t')
  return split_text[0:1],'starttoken '+split_text[1:2]+' endtoken'

In [ ]:
init_dataset=text_dataset.map(separator)

In [ ]:
for i in init_dataset.take(3):
  print(i)

In [ ]:
english_training_data=init_dataset.map(lambda x,y:x)
english_vectorize_layer.adapt(english_training_data)

In [ ]:
french_training_data=init_dataset.map(lambda x,y:y)
french_vectorize_layer.adapt(french_training_data)

In [ ]:
def vectorizer(inputs,output):
  return {'input_1':english_vectorize_layer(inputs['input_1']),
          'input_2':french_vectorize_layer(inputs['input_2'])},french_vectorize_layer(output)

In [ ]:
dataset=split_dataset.map(vectorizer)

In [ ]:
for i in dataset.take(1):
  print(i)

In [ ]:
dataset=dataset.shuffle(2048).unbatch().batch(BATCH_SIZE).prefetch(buffer_size=tf.data.AUTOTUNE)

In [ ]:
NUM_BATCHES=int(200000/BATCH_SIZE)

In [ ]:
train_dataset=dataset.take(int(0.9*NUM_BATCHES))
val_dataset=dataset.skip(int(0.9*NUM_BATCHES))

In [ ]:
NUM_UNITS = 256

In [ ]:
input = Input(shape=(ENGLISH_SEQUENCE_LENGTH,), dtype="int64", name="input_1")
x=Embedding(VOCAB_SIZE, EMBEDDING_DIM, )(input)
encoded_input=Bidirectional(GRU(NUM_UNITS), )(x)

shifted_target=Input(shape=(FRENCH_SEQUENCE_LENGTH,), dtype="int64", name="input_2")
x=Embedding(VOCAB_SIZE,EMBEDDING_DIM,)(shifted_target)
x = GRU(NUM_UNITS*2, return_sequences=True)(x, initial_state=encoded_input)

x = Dropout(0.5)(x)
target=Dense(VOCAB_SIZE,activation="softmax")(x)
seq2seq_gru=Model([input,shifted_target],target)
seq2seq_gru.summary()

In [ ]:
class BLEU(tf.keras.metrics.Metric):
  def __init__(self, name='bleu_score'):
    super(BLEU, self).__init__()
    self.bleu_score = 0

  def update_state(self, y_true, y_pred, sample_weight=None):
    y_pred = tf.argmax(y_pred, axis=-1)
    self.bleu_score = 0
    for i,j in zip(y_pred, y_true):
      tf.autograph.experimental.set_loop_options() # add this line beacuse after removing element from j it give error.
      total_word = tf.math.count_nonzero(i)
      total_matches = 0
      for word in i:
        if word==0:
          break
        for q in range(len(j)):
          if j[q] == 0:
            break
          if j[q] == word:
            total_matches += 1
            j = tf.boolean_mask(j, [False if y == q else True for y in range(len(j))])
            break
    self.bleu_score += total_matches/total_word

  def result(self):
    return self.bleu_score/BATCH_SIZE

In [ ]:
seq2seq_gru.compile(
    loss=tf.keras.losses.SparseCategoricalCrossentropy(),
    optimizer=tf.keras.optimizers.Adam(5e-4), metrics=[BLEU()])

In [ ]:
history=seq2seq_gru.fit(
    train_dataset,
    validation_data=val_dataset,
    epochs=15,)

In [ ]:
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('model_loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'val'], loc='upper left')
plt.show()

In [ ]:
plt.plot(history.history['accuracy'])
plt.plot(history.history['val_accuracy'])

plt.title('model_accuracy')
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.legend(['train', 'val'], loc='upper left')
plt.show()

In [ ]:
index_to_word={x:y for x, y in zip(range(len(french_vectorize_layer.get_vocabulary())),
                                   french_vectorize_layer.get_vocabulary())}

In [ ]:
def translator(english_sentence):
  tokenized_english_sentence=english_vectorize_layer([english_sentence])
  shifted_target = 'starttoken'

  for i in range(FRENCH_SEQUENCE_LENGTH):
    tokenized_shifted_target=french_vectorize_layer([shifted_target])

    output = seq2seq_gru.predict([tokenized_english_sentence,tokenized_shifted_target])
    french_word_imdex = tf.argmax(output, axis=-1)[0][i].numpy()
    current_word = index_to_word[french_word_imdex]

    if current_word == 'endtoken':
      break

    shifted_target += ' ' + current_word

  return shifted_target[11:]

In [ ]:
translator('What makes you think that is not true?')

In [ ]:
class Encoder(tf.keras.Model):
  def __init__(self, vocab_size, embedding_dim, units):
    super(Encoder, self).__init__()
    self.embedding_dim = embedding_dim
    self.units = units
    self.vocab_size = vocab_size

  def build(self, input_shape):
    self.embedding = Embedding(self.vocab_size, self.embedding_dim)
    self.lstm = LSTM(self.units, return_sequences=True)

  def call(self, x):
    x = self.embedding(x)
    output = self.lstm(x)
    return output

In [ ]:
HIDDEN_UNITS = 256
EMBEDDING_DIM = 256
encoder = Encoder(VOCAB_SIZE, EMBEDDING_DIM, HIDDEN_UNITS)
encoder_output = encoder(tf.zeros((128, 64)))
encoder_output.shape

In [ ]:
class BahdanauAttention(tf.keras.Model):
  def __init__(self, units):
    super(BahdanauAttention, self).__init__()
    self.units = units

  def build(self, input_shape):
    self.w_1 = Dense(self.units)
    self.w_2 = Dense(self.units)
    self.w = Dense(1)

  def call(self, prev_dec_state, enc_state):
    scores = self.w(
        tf.nn.tanh(
            self.w_1(tf.expand_dims(prev_dec_state, -2)) + self.w_2(enc_state)
        ))
    attention_weights = tf.nn.softmax(scores, axis=1)
    context_vector = attention_weights*enc_state
    context_vector = tf.reduce_sum(context_vector, axis=1)
    return context_vector

In [ ]:
bahdanau_attention = BahdanauAttention(256)
context_vector = bahdanau_attention(tf.zeros((128, 32)), tf.zeros((128, 64, 32)))
context_vector.shape

In [ ]:
class Decoder(tf.keras.Model):
  def __init__(self,vocab_size,embedding_dim,dec_units,sequence_length):
    super(Decoder,self).__init__()
    self.embedding_dim=embedding_dim
    self.vocab_size=vocab_size
    self.dec_units=dec_units
    self.sequence_length=sequence_length

  def build(self,input_shape):
    self.dense=Dense(self.vocab_size,activation="softmax")
    self.gru=GRU(
        self.dec_units,return_sequences=True,return_state=True)
    self.attention=BahdanauAttention(self.dec_units)
    self.embedding=Embedding(self.vocab_size,self.embedding_dim)

  def call(self,x,hidden,shifted_target):
    outputs=[]
    context_vectors=[]
    shifted_target=self.embedding(shifted_target)

    for t in range(0,self.sequence_length):
      context_vector=self.attention(hidden,x)
      dec_input=context_vector+shifted_target[:,t]
      output,hidden=self.gru(tf.expand_dims(dec_input,1))
      outputs.append(output[:,0])

    outputs=tf.convert_to_tensor(outputs)
    outputs=tf.transpose(outputs, perm=[1,0,2])

    outputs=self.dense(outputs)
    return outputs

In [ ]:
class Decoder(tf.keras.Model):
  def __init__(self,vocab_size,embedding_dim,dec_units,sequence_length):
    super(Decoder,self).__init__()
    self.embedding_dim=embedding_dim
    self.vocab_size=vocab_size
    self.dec_units=dec_units
    self.sequence_length=sequence_length

  def build(self,input_shape):
    self.dense=Dense(self.vocab_size,activation="softmax")
    self.gru=GRU(
        self.dec_units,return_sequences=True,return_state=True)
    self.attention=BahdanauAttention(self.dec_units)
    self.embedding=Embedding(self.vocab_size,self.embedding_dim)

  def call(self,x,hidden,shifted_target):
    outputs=[]
    context_vectors=[]
    shifted_target=self.embedding(shifted_target)

    for t in range(0,self.sequence_length):
      context_vector=self.attention(hidden,x)
      dec_input=context_vector+shifted_target[:,t]
      # The original code used tf.expand_dims(dec_input,1) which caused the error
      # Use tf.reshape to specify the shape explicitly
      output,hidden=self.gru(tf.reshape(dec_input, [128, 1, dec_input.shape[1]]))
      outputs.append(output[:,0])

    outputs=tf.convert_to_tensor(outputs)
    outputs=tf.transpose(outputs, perm=[1,0,2])

    outputs=self.dense(outputs)
    return outputs

In [ ]:
decoder=Decoder(VOCAB_SIZE,EMBEDDING_DIM,HIDDEN_UNITS,FRENCH_SEQUENCE_LENGTH)
outputs=decoder(encoder_output,tf.zeros([128,HIDDEN_UNITS]),tf.zeros([128,64]))
outputs.shape

TypeError: Exception encountered when calling Decoder.call().

[1mCould not automatically infer the output shape / dtype of 'decoder_5' (of type Decoder). Either the `Decoder.call()` method is incorrect, or you need to implement the `Decoder.compute_output_spec() / compute_output_shape()` method. Error encountered:

Exception encountered when calling GRU.call().

[1mlen is not well defined for a symbolic Tensor (gru_5_1/Squeeze:0). Please call `x.shape` rather than `len(x)` for shape information.[0m

Arguments received by GRU.call():
  • sequences=tf.Tensor(shape=(128, 1, 256), dtype=float32)
  • initial_state=None
  • mask=None
  • training=False[0m

Arguments received by Decoder.call():
  • args=('<KerasTensor shape=(None, 64, 256), dtype=float32, sparse=False, name=keras_tensor>', 'tf.Tensor(shape=(128, 256), dtype=float32)', 'tf.Tensor(shape=(128, 64), dtype=float32)')
  • kwargs=<class 'inspect._empty'>

In [ ]:
### ENCODER
input = Input(shape=(ENGLISH_SEQUENCE_LENGTH,), dtype="int64", name="input_1")
encoder=Encoder(VOCAB_SIZE,EMBEDDING_DIM,HIDDEN_UNITS)
encoder_output=encoder(input)

### DECODER
shifted_target=Input(shape=(FRENCH_SEQUENCE_LENGTH,), dtype="int64", name="input_2")
decoder=Decoder(VOCAB_SIZE,EMBEDDING_DIM,HIDDEN_UNITS,FRENCH_SEQUENCE_LENGTH)
decoder_output,attention_weightss=decoder(encoder_output,tf.zeros([1,HIDDEN_UNITS]),shifted_target)

### OUTPUT
bahdanau=Model([input,shifted_target],decoder_output)
bahdanau.summary()

In [ ]:
bahdanau.compile(
    loss=tf.keras.losses.SparseCategoricalCrossentropy(),
    optimizer=tf.keras.optimizers.Adam(5e-4),)

In [ ]:
history=bahdanau.fit(
    train_dataset,
    validation_data=val_dataset,
    epochs=30,)

In [ ]:
def positional_encoding(model_size,SEQUENCE_LENGTH):
  output=[]
  for pos in range(SEQUENCE_LENGTH):
    PE=np.zeros((model_size))
    for i in range(model_size):
      if i%2==0:
        PE[i]=np.sin(pos/(10000**(i/model_size)))
      else:
        PE[i]=np.cos(pos/(10000**((i-1)/model_size)))
    output.append(tf.expand_dims(PE,axis=0))
  out=tf.concat(output,axis=0)
  out=tf.expand_dims(out,axis=0)
  return tf.cast(out,dtype=tf.float32)

In [ ]:
class Embeddings(tf.keras.Layer):
  def __init__(self, sequence_length, vocab_size, embed_dim,):
    super(Embeddings, self).__init__()
    self.token_embeddings=Embedding(
        input_dim=vocab_size, output_dim=embed_dim)
    self.sequence_length = sequence_length
    self.vocab_size = vocab_size
    self.embed_dim = embed_dim

  def call(self, inputs):
    embedded_tokens = self.token_embeddings(inputs)
    embedded_positions=positional_encoding(
        self.embed_dim,self.sequence_length)
    return embedded_tokens + embedded_positions

  def compute_mask(self, inputs, mask=None):
    return tf.math.not_equal(inputs, 0)

In [ ]:
class CustomSelfAttention(tf.keras.Layer):
  def __init__(self, model_size):
    super(CustomMultiHeadAttention, self).__init__()
    self.model_size = model_size

  def call(self, query, key, value, masking):
    score = tf.matmul(query, key, transpose_b=True)

    score /= tf.math.sqrt(tf.cast(self.model_size, tf.float32))

    masking = tf.cast(masking, dtype=tf.float32)
    score += (1.-masking) * -1e9
    attention = tf.nn.softmax(score, axis=-1) * masking
    head = tf.matmul(attention, value)

    return head

In [ ]:
class CustomMultiHeadAttention(tf.keras.Layer):
  def __init__(self, num_heads, key_dim):
    super(CustomMultiHeadAttention, self).__init__()
    self.num_heads = num_heads
    self.dense_q = [Dense(key_dim) for _ in range(num_heads)]
    self.dense_k = [Dense(key_dim) for _ in range(num_heads)]
    self.dense_v = [Dense(key_dim) for _ in range(num_heads)]
    self.dense_o = Dense(key_dim)
    self.self_attention = CustomSelfAttention(key_dim)

  def call(self, query, key, value, attention_mask):
    heads = []

    for i in range(self.num_heads):
      head = self.self_attention(self.dense_q[i](query), self.dense_k[i](key), self.dense_v[i](value), attention_mask)

      heads.append(head)

    heads = tf.concat(heads, axis=2)
    heads = self.dense_o(heads)

    return heads

In [ ]:
class TransformerEncoder(tf.keras.Layer):
    def __init__(self, embed_dim, dense_dim, num_heads,):
        super(TransformerEncoder, self).__init__()
        self.embed_dim = embed_dim
        self.dense_dim = dense_dim
        self.num_heads = num_heads
        self.attention = CustomMultiHeadAttention(
            num_heads=num_heads, key_dim=embed_dim,
        )
        self.dense_proj=tf.keras.Sequential(
            [Dense(dense_dim, activation="relu"),
             Dense(embed_dim),]
        )
        self.layernorm_1 = LayerNormalization()
        self.layernorm_2 = LayerNormalization()
        self.supports_masking = True

    def call(self, inputs, mask=None):

      if mask is not None:
        mask = tf.cast(
            mask[:,tf.newaxis, :], dtype="int32")
        T = tf.shape(mask)[2]
        padding_mask = tf.repeat(mask,T,axis=1)
      attention_output = self.attention(
          query=inputs, key=inputs,value=inputs,
          attention_mask=padding_mask
      )

      proj_input = self.layernorm_1(inputs + attention_output)
      proj_output = self.dense_proj(proj_input)
      return self.layernorm_2(proj_input + proj_output)

In [ ]:
class TransformerDecoder(tf.keras.Layer):
  def __init__(self, embed_dim, latent_dim, num_heads,):
    super(TransformerDecoder, self).__init__()
    self.embed_dim = embed_dim
    self.latent_dim = latent_dim
    self.num_heads = num_heads
    self.attention_1=CustomMultiHeadAttention(
        num_heads=num_heads, key_dim=embed_dim
    )
    self.attention_2=CustomMultiHeadAttention(
        num_heads=num_heads, key_dim=embed_dim
    )
    self.dense_proj = tf.keras.Sequential(
        [Dense(latent_dim, activation="relu"),Dense(embed_dim),]
    )
    self.layernorm_1=LayerNormalization()
    self.layernorm_2=LayerNormalization()
    self.layernorm_3=LayerNormalization()
    self.supports_masking = True
  def call(self, inputs, encoder_outputs, enc_mask, mask=None):


    if mask is not None:
      causal_mask=tf.linalg.band_part(
        tf.ones([tf.shape(inputs)[0],
                 tf.shape(inputs)[1],
                 tf.shape(inputs)[1]],dtype=tf.int32),-1,0)
      mask = tf.cast(
          mask[:,tf.newaxis, :], dtype="int32")
      enc_mask = tf.cast(
          enc_mask[:,tf.newaxis, :], dtype="int32")
      T = tf.shape(mask)[2]
      padding_mask = tf.repeat(mask,T,axis=1)
      cross_attn_mask = tf.repeat(enc_mask,T,axis=1)
      combined_mask=tf.minimum(padding_mask,causal_mask)

    attention_output_1 = self.attention_1(
        query=inputs,key=inputs,value=inputs,
        attention_mask=combined_mask,

    )

    out_1 = self.layernorm_1(inputs + attention_output_1)

    attention_output_2, scores = self.attention_2(
        query=out_1,key=encoder_outputs,value=encoder_outputs,
        attention_mask=cross_attn_mask,
        return_attention_scores=True
    )
    out_2 = self.layernorm_2(out_1 + attention_output_2)

    proj_output = self.dense_proj(out_2)
    return self.layernorm_3(out_2 + proj_output), scores

In [ ]:
EMBEDDING_DIM=512
D_FF=2048
NUM_HEADS=8
NUM_LAYERS=1
NUM_EPOCHS=10
attention_scores = {}

In [ ]:
encoder_inputs=Input(shape=(None,), dtype="int64", name="input_1")
emb = Embeddings(ENGLISH_SEQUENCE_LENGTH,VOCAB_SIZE,EMBEDDING_DIM)
x = emb(encoder_inputs)
enc_mask = emb.compute_mask(encoder_inputs)

for _ in range(NUM_LAYERS):
  x=TransformerEncoder(EMBEDDING_DIM,D_FF,NUM_HEADS)(x)
encoder_outputs=x

decoder_inputs=Input(shape=(None,), dtype="int64", name="input_2")

x = Embeddings(FRENCH_SEQUENCE_LENGTH,VOCAB_SIZE,EMBEDDING_DIM)(decoder_inputs)
for i in range(NUM_LAYERS):
  x,scores=TransformerDecoder(EMBEDDING_DIM,D_FF,NUM_HEADS)(x, encoder_outputs,enc_mask)
  attention_scores[f'decoder_layer{i+1}_block2'] = scores

decoder_outputs=Dense(VOCAB_SIZE, activation="softmax")(x)

attention_score_model = tf.keras.Model(
    [encoder_input, decoder_input], attention_scores, name="attention_score_model"
)

transformer = tf.keras.Model(
    [encoder_inputs, decoder_inputs], decoder_outputs, name="transformer"
)
transformer.summary()

ValueError: A KerasTensor cannot be used as input to a TensorFlow function. A KerasTensor is a symbolic placeholder for a shape and dtype, used when constructing Keras Functional models or Keras Functions. You can only use it as input to a Keras layer or a Keras operation (from the namespaces `keras.layers` and `keras.operations`). You are likely doing something like:

```
x = Input(...)
...
tf_fn(x)  # Invalid.
```

What you should do instead is wrap `tf_fn` in a layer:

```
class MyLayer(Layer):
    def call(self, x):
        return tf_fn(x)

x = MyLayer()(x)
```


In [ ]:
class Scheduler(LearningRateSchedule):
  def __init__(self, d_model, warmup_steps):
    super(Scheduler, self).__init__()
    self.d_model = tf.cast(d_model, tf.float64)
    self.warmup_steps = tf.cast(warmup_steps, dtype=tf.float64)

  def __call__(self, step):
    step = tf.cast(step, dtype=tf.float64)
    return (self.d_model**(-0.5))*tf.math.minimum(step**(-0.5), step * (self.warmup_steps ** -1.5))

In [ ]:
WARM_UP_STEPS = 4000
lr_scheduled = Scheduler(EMBEDDING_DIM, WARM_UP_STEPS)

In [ ]:
transformer.compile(
    loss=tf.keras.losses.SparseCategoricalCrossentropy(),
    optimizer = Adam(lr_scheduled, beta_1=0.9, beta_2=0.98, epsilon=1e-9),)

In [ ]:
history=transformer.fit(
    train_dataset,
    validation_data=val_dataset,
    epochs=10)

In [ ]:
def visualize(english_sentence):
  tokenized_english_sentence=english_vectorize_layer([english_sentence])
  shifted_target = 'starttoken jal lai fait tres bien'

  tokenized_shifted_target=french_vectorize_layer([shifted_target])
  attention_weights = attention_score_model.predict([tokenized_english_sentence,tokenized_shifted_target])

  return attention_weights

out = visualize("I did it very well")

In [ ]:
fig = plt.figure(figsize=(8,6))
plt.imshow(out['decoder_layer1_block2'][0][0][0:10,0:10])
plt.title("Attention score")
plt.show()

In [ ]:
plt.figure(figsize = (12,12))

for i in range(NUM_HEADS):
  ax = plt.subplot(2,4, i+1)

  plt.imshow(out['decoder_layer1_block2'][0][i][0:10,0:10])
  plt.title("Attention Scores for head:->"+str(i+1))